In [1]:
from mexc_sdk import Spot
import os 
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
symbol = input("Symbol = ")
folder_save = f"../../data/{symbol}"

start_date = input(f"Start date = (YYYY-MM-DD)").strip()
start_date = start_date.split("-")
start_date = [int(item) for item in start_date]

to_date = input(f"To date = (YYYY-MM-DD)").strip()
to_date = to_date.split("-")
to_date = [int(item) for item in to_date]

is_get_now = input("Get now? (y/n)").strip().lower()[0] == "y"

interval = input("""interval = 
                 
                "1m" - "1 minute"
                "5m" - "5 minute"
                "15m" - "15 minute"
                "30m" - "30 minute"
                "60m" - "60 minute"
                "4h" - "4 hour"
                "1d" - "1 day"
                "1W" - "1 week"
                "1M" - "1 month"
                 """).strip()

In [3]:
load_dotenv(dotenv_path="./apikey.txt")

True

In [4]:
access_key = os.getenv("access_key")
secret_key = os.getenv("secret_key")

In [5]:
spot = Spot(api_key=access_key, api_secret=secret_key)

In [6]:
data_from_mexc = spot.klines(symbol.upper() + "USDT", interval=interval, 
    # options={
    #     "startTime": start_date,
    #     "endTime": to_date
    # }
    )

In [7]:
df_origin_mexc = pd.DataFrame(data_from_mexc)

In [8]:
df_origin_mexc.columns

RangeIndex(start=0, stop=8, step=1)

In [9]:
df_origin_mexc["Date"] = pd.to_datetime(df_origin_mexc.iloc[:, 0], unit="ms")

In [10]:
mapping_prices = {1: "Open",
                  2: "High",
                  3: "Low",
                  4: "Close",
                  5: "Volume", }

In [11]:
for index in mapping_prices:
    df_origin_mexc[mapping_prices[index]] = df_origin_mexc.iloc[:, index]

In [12]:
df_from_mexc = df_origin_mexc[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [13]:
now = datetime.now().date()

In [14]:
now

datetime.date(2024, 11, 27)

In [15]:
df_from_mexc

,Open,High,Low,Close,Volume,Date
0,30232,30336.96,29660.13,30138.01,4253.923631,2023-07-17
1,30138.01,30240.47,29518.02,29859.13,4280.493334,2023-07-18
2,29859.13,30188,29762.93,29909.21,4293.324296,2023-07-19
3,29909.21,30417.46,29578,29800,4916.07642,2023-07-20
4,29800,30061.7,29726.34,29901.71,3547.116708,2023-07-21
...,...,...,...,...,...,...
495,98892,98908.86,97136,97672.4,3694.46996592,2024-11-23
496,97672.4,98555,95734.78,97900.01,3244.43903062,2024-11-24
497,97900.01,98871.8,92611.86,93018.97,7432.21817185,2024-11-25
498,93018.97,94972.4,90791.13,91965.16,6497.88820608,2024-11-26


In [16]:
if not is_get_now:
    df_from_mexc = df_from_mexc[df_from_mexc["Date"] != str(now)]

In [18]:
old_df = pd.read_csv(f"{folder_save}/{symbol.lower()}_to_now_{interval.lower()}.csv")
old_df["Date"] = pd.to_datetime(old_df["Date"])

# min_date_in_mexc = str(df_from_mexc["Date"].min())
# old_df = old_df[old_df["Date"] < min_date_in_mexc]



In [19]:
old_df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,2.105680e+07,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,3.448320e+07,2014-09-18
2,424.102997,427.834991,384.532013,394.795990,3.791970e+07,2014-09-19
3,394.673004,423.295990,389.882996,408.903992,3.686360e+07,2014-09-20
4,408.084991,412.425995,393.181000,398.821014,2.658010e+07,2014-09-21
...,...,...,...,...,...,...
3718,94284.000000,98987.000000,94040.000000,98317.120000,7.602617e+03,2024-11-21
3719,98317.120000,99588.010000,97132.000000,98892.000000,4.843682e+03,2024-11-22
3720,98892.000000,98908.860000,97136.000000,97672.400000,3.694470e+03,2024-11-23
3721,97672.400000,98555.000000,95734.780000,97900.010000,3.244439e+03,2024-11-24


In [20]:
old_df = old_df[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [21]:
old_df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,2.105680e+07,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,3.448320e+07,2014-09-18
2,424.102997,427.834991,384.532013,394.795990,3.791970e+07,2014-09-19
3,394.673004,423.295990,389.882996,408.903992,3.686360e+07,2014-09-20
4,408.084991,412.425995,393.181000,398.821014,2.658010e+07,2014-09-21
...,...,...,...,...,...,...
3718,94284.000000,98987.000000,94040.000000,98317.120000,7.602617e+03,2024-11-21
3719,98317.120000,99588.010000,97132.000000,98892.000000,4.843682e+03,2024-11-22
3720,98892.000000,98908.860000,97136.000000,97672.400000,3.694470e+03,2024-11-23
3721,97672.400000,98555.000000,95734.780000,97900.010000,3.244439e+03,2024-11-24


In [22]:
max_date_in_old_df = str(old_df["Date"].max())

In [23]:
max_date_in_old_df

'2024-11-25 00:00:00'

In [24]:
df_from_mexc

,Open,High,Low,Close,Volume,Date
0,30232,30336.96,29660.13,30138.01,4253.923631,2023-07-17
1,30138.01,30240.47,29518.02,29859.13,4280.493334,2023-07-18
2,29859.13,30188,29762.93,29909.21,4293.324296,2023-07-19
3,29909.21,30417.46,29578,29800,4916.07642,2023-07-20
4,29800,30061.7,29726.34,29901.71,3547.116708,2023-07-21
...,...,...,...,...,...,...
494,98317.12,99588.01,97132,98892,4843.68223563,2024-11-22
495,98892,98908.86,97136,97672.4,3694.46996592,2024-11-23
496,97672.4,98555,95734.78,97900.01,3244.43903062,2024-11-24
497,97900.01,98871.8,92611.86,93018.97,7432.21817185,2024-11-25


In [25]:
df_today = df_from_mexc[df_from_mexc["Date"] > max_date_in_old_df]


In [26]:
df_today

,Open,High,Low,Close,Volume,Date
498,93018.97,94972.4,90791.13,91965.16,6497.88820608,2024-11-26


In [27]:
old_df["Date"] = old_df["Date"].dt.date
df_today["Date"] = df_today["Date"].dt.date
df = pd.concat([old_df, df_today], ignore_index=True)
df["Date"] = pd.to_datetime(df["Date"])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_39048\1586841953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_today["Date"] = df_today["Date"].dt.date


In [28]:
df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,21056800.0,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,34483200.0,2014-09-18
2,424.102997,427.834991,384.532013,394.79599,37919700.0,2014-09-19
3,394.673004,423.29599,389.882996,408.903992,36863600.0,2014-09-20
4,408.084991,412.425995,393.181,398.821014,26580100.0,2014-09-21
...,...,...,...,...,...,...
3719,98317.12,99588.01,97132.0,98892.0,4843.682236,2024-11-22
3720,98892.0,98908.86,97136.0,97672.4,3694.469966,2024-11-23
3721,97672.4,98555.0,95734.78,97900.01,3244.439031,2024-11-24
3722,97900.01,98871.8,92611.86,93018.97,7432.218172,2024-11-25


In [29]:
symbol = symbol.replace(".", "_").lower()

In [34]:
min_date = df["Date"].min().date()
max_date = df["Date"].max().date()

os.makedirs(folder_save, exist_ok=True)
file_name = f"{folder_save}/{symbol}_to_now_{interval.lower()}.csv"

In [35]:
file_name = file_name.replace("-", "_")

In [36]:
file_name

'../../data/btc/btc_to_now_1d.csv'

In [37]:
df.to_csv(file_name, index=False)